## Get Fulltexts

### Imports

In [21]:
import gzip
import json
from pathlib import Path
import tarfile


import pandas as pd

### Folders

In [24]:
data_folder = Path("data/")
metadata = data_folder / Path("metadata.csv")
cord_19_fulltexts = data_folder / Path("2020-07-16/document_parses.tar.gz")

### Mappings

read metadata table and get `cord_uid's` and `path`

In [4]:
md = pd.read_csv(metadata)

/tmp/ipykernel_18714/4265034585.py:1: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv(metadata)


In [19]:
cord_file_names = md.dropna(axis=0, subset=["pdf_json_files"])[["pdf_json_files", "cord_uid"]].to_dict(orient="records")
mappings = []
for cord_dict in cord_file_names:
    cord_uid = cord_dict["cord_uid"]
    path_str = cord_dict["pdf_json_files"]
    cord_dict["pdf_json_files"] = [path.lstrip().strip() for path in path_str.split(";")]
    mappings.append(cord_dict)

save mappings under `data/file_mappings.json`

In [22]:
mappings_path = data_folder / Path("file_mappings.json")

if not mappings_path.is_file():
    json.dump(mappings, open(mappings_path, "w"))

### Get Fulltexts

go trought mappings and collect fulltext then save under `data/fulltexts.json.gz`

In [31]:
tar = tarfile.open(cord_19_fulltexts)

for cord_dict in mappings:
    cord_uid = cord_dict["cord_uid"]
    path_list = cord_dict["pdf_json_files"]
    for path in path_list:
        try:
            tar.extract(path, data_folder)
        except Exception as err:
            print(err, path)
tar.close()

KeyboardInterrupt: 